In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy.integrate import quad
from scipy.special import hyp2f1

from StringIO import StringIO 
%matplotlib inline


In [5]:
data=pd.read_csv('mock_cluster_4.csv')
data.rename(columns={'Unnamed: 0': 'radius2D', '0':'shellId', '1':'x', '2':'y', '3':'z', '4':'v'},inplace=True)
data.radius2D = np.sqrt(np.power(data.x,2)+np.power(data.y,2))
data.describe()


,radius2D,shellId,x,y,z,v
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,154.205904,4.500000,4.662470,-7.206179,3.370333,6.829582
std,222.522683,2.873719,192.513279,190.222250,188.251102,295.177409
min,0.282092,0.000000,-885.504044,-914.696833,-935.733984,-1800.790183
25%,7.787780,2.000000,-21.543062,-22.828401,-19.806140,-99.982980
50%,47.379177,4.500000,0.260367,-0.425345,0.406701,2.582063
75%,212.501908,7.000000,28.387367,20.662865,24.898741,119.634603
max,982.440911,9.000000,939.847104,952.085239,864.004359,1979.414230


In [6]:
N_total =  data['x'].count()
del data['x']
del data['y']
del data['z']
del data['shellId']
print N_total

1000


In [7]:
# here created the grid cell boundaries for our entries 
def get_the_grid(arr):
    # 1D array
    # has to be sorted!!! 
    arr = np.array(arr)
    grid = (arr[1:]+arr[:-1])/2.
    #print grid.shape
    #print arr[:-1].shape
    grid = np.append(2.*arr[0]-grid[0], grid)
    grid = np.append(grid, 2.*arr[-1]-grid[-1])
    #grid = np.append(grid, 2.*arr[arr.shape[0]-1]-grid[-1])
    grid_low = grid[:-1]
    grid_up = grid[1:]
    return grid_low, grid_up                                                                                                                                                   

data = data.sort(['radius2D'])
grid_low, grid_up = get_the_grid(data['radius2D'])
if grid_low[0]<0:
    grid_low[0]=0.
#print grid_low.shape, grid_up.shape, data.radius2D.shape
data['dr_in']= grid_low
data['dr_out'] = grid_up

data = data.sort(['v'])
grid_low, grid_up = get_the_grid(data['v'])
data['dv_in']= grid_low
data['dv_out'] = grid_up


#data_vel_sort = data.sort(['v'])

In [12]:
data  =  data.sort(['radius2D'])

In [66]:
def nu_f(z, r2D, nu_0, k):
    return nu_0*z*np.power((np.power(r2D,2) + z**2),-k/2.)*np.power((z**2/np.power(r2D,2) +1), k/2.)* \
           hyp2f1(0.5, k/2., 1.5, -(z**2/np.power(r2D,2)))

def nu_int(z, r2D, nu_0, k):
    return nu_0*2.*np.pi*r2D/np.power((np.power(r2D,2)+z**2),k/2.)
    
def sig_f_sq(z, r2D, sig_0, m):
    nu_0 = sig_0 # change this later
    k = m
    return sig_0**2/np.power(np.power(z,2)+np.power(r2D,2)  , m) # because we use sigma square, cause thats how gaussians add

def sig_f(z, r2D, sig_0, m):
    nu_0 = sig_0 # change this later
    k = m
    return sig_0/np.power(np.power(z,2)+np.power(r2D,2)  , m/2.) # because we use sigma square, cause thats how gaussians add


def vel_spread(v, sigma2D, v_mean, Ndr):
    return Ndr/(np.sqrt(2.*np.pi)*sigma2D)*np.exp(-np.power((v-v_mean),2)/(2.*np.power(sigma2D,2)))

            

def log_likelihood_2(sig_0, m, nu_0, k, v_mean=6.829582):
    # note that everything here is performed only on cells that actually have galaxies in them
    z_range = np.sqrt(1000.**2 - np.power(np.array(data.radius2D),2))
    N_r = np.array([quad(nu_int, -z, z, args=(r2D, nu_0, k)) for z, r2D in zip(z_range, np.array(data.radius2D))])
    N_r = N_r[:,0]*(data.dr_out-data.dr_in)
    print np.sum(N_r)
    #sigma_2D = quad(sig_f, data.dr_in, data.dr_out, args=(data.radius2D, sig_0, m))# another scarry integral # projected sigma in that grid cell
    sigma_2D_sq = np.sqrt([quad(sig_f_sq, -z, z, args=(r2D, sig_0, m)) for z, r2D in zip(z_range, np.array(data.radius2D))])
    sigma_2D_sq = sigma_2D_sq[:,0]
    sigma_2D = np.array([quad(sig_f, -z, z, args=(r2D, sig_0, m)) for z, r2D in zip(z_range, np.array(data.radius2D))])
    sigma_2D = sigma_2D[:,0]/(2.*z_range)

    print sigma_2D

    #N_v =  quad(vel_spread, data.dv_in, data.dv_out, args=(sigma2D, v_mean, N_r))# integral from dv1 to dv2 of gaussian with sigma 2d, that is normalized by N_r # number of galaxies in the dv range
    N_v = np.array([vel_spread(np.array(data.v), sig, v_mean, nr)*(data.dv_out-data.dv_in) for sig,nr in zip(sigma_2D, N_r)])
    print N_r.shape, N_v.shape
    #N_v = N_v*(data.dv_out-data.dv_in)
    print "all detections:", np.sum(N_v)
    #return (-1.*np.sum(N_det)*(np.sum(np.log(N_det))))

def dNdrdv(z, r2D, v, sig_0, m, nu_0, k, v_mean=6.829582):
    sigma = sig_f(z, r2D, sig_0, m)
    return 2.*np.pi*sig_0**2/np.power(np.power(z,2)+np.power(r2D,2)  , m)*\
            1./(np.sqrt(2.*np.pi)*sigma)*np.exp(-np.power((v-v_mean),2)/(2.*np.power(sigma,2)))

def log_likelihood(sig_0, m, nu_0, k, v_mean=6.829582):
    # note that everything here is performed only on cells that actually have galaxies in them
    z_range = np.sqrt(1000.**2 - np.power(np.array(data.radius2D),2))
    
    dN = np.array([quad(dNdrdv, -z, z, args=(r2D, v, sig_0, m, nu_0, k)) for z,v, r2D in zip(z_range, np.array(data.v),np.array(data.radius2D))])
    print dN.shape
    print "all detections:", np.sum(dN[:,0]/((np.amax(data.radius2D)-np.amin(data.radius2D))*(np.amax(data.v)-np.amin(data.v))))
                                               
    

    
def log_prior(m, b):
    return 0.
    
def straight_line_log_posterior(x,y,sigmay, m,b):
    return (log_likelihood(x,y,sigmay, m,b) + log_prior(m, b))
# initial parameters (from actual distribution used)
nu_0_th = 11.5200189254
k_th = 3.
sig_0_th = 1000.
m_th = 0.2 
v_mean =6.829582 #52.580055# taken from the data
log_likelihood(sig_0_th, m_th, nu_0_th, k_th)

(1000, 2)
all detections: 307.346756965
